In [1]:
# !pip install sklearn

In [4]:
import pandas as pd
import sklearn

In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn import svm

In [5]:
df = pd.read_csv('dataset_moyennes_matchs.csv')

### 1. Suppression des variables inutiles

In [6]:
# suppression des cotes et nb de goals dont on ne s'occupe pas pour le moment
df = df.drop(['FTHG','FTAG','B365H','B365D','B365A','LBH','LBD','LBA','PSH','PSD','PSA','WHH','WHD','WHA','VCH','VCD','VCA','PSCH','PSCD','PSCA'], axis=1)
# suppression de l'ancien idx
df = df.drop(['Unnamed: 0'], axis=1)
# suppression des variables informatives non nécessaires pour la modélisation
df = df.drop(['match_id','season','date','away_team_id','away_team_name','home_team_id','home_team_name'],axis=1)

### 2. Suppression des variables non significatives

- Observation de la variance

In [18]:
# ! pip install pandas-profiling

In [ ]:
# test de l'outil pandas profiling

In [14]:
from pandas_profiling import ProfileReport
report = ProfileReport(df)

In [21]:
report.to_notebook_iframe()

In [ ]:
# le fichier html peut ensuite être ouvert pour afficher le rapport
report.to_file("rapport_variables.html")

In [28]:
report.to_file('variables.json')

Export report to file: 100%|██████████| 1/1 [00:00<00:00,  4.46it/s]


In [29]:
# Vérification dans le rapport des variables taggées avec "UNIQUE", "HIGH CARDINALITY", ou "UNIFORM", indicatrices d'une variance trop/faible/élevée ou de valeurs manquantes
# Le rapport identifie away_pass comme "unique"
# "[UNIQUE] alert on column away_pass"
# étant donné que la variable home_pass n'est pas signalée, je ne supprime pas

- PCA

#### 3. Isolement variable target

In [7]:
target = df['FTR']
data = df.drop(['FTR'],axis=1)

In [8]:
# remplacement par classes numériques
target = target.replace(['H','A','D'],[1,2,3])

### 4. Jeu d'entraînement / test

In [ ]:
# pour l'instant, séparation sur tout le jeu sans distinguer les périodes, voir si le module https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.TimeSeriesSplit.html serait pertinent ici

In [68]:
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2)

#### 5. Standardisation

In [69]:
scaler = preprocessing.StandardScaler().fit(X_train)

X_train_scaled = scaler.transform(X_train)

In [70]:
X_test_scaled = scaler.transform(X_test)

#### 6. Modèles

Recherche de modèles de classification multi-classes performants pour le jeu de données

- SVM

In [71]:
from sklearn.svm import SVC

In [72]:
# ajout d'un OneVsRestClassifier car les numéros des classes ne correspondent pas à un ordre spécifique
from sklearn.multiclass import OneVsRestClassifier
ovr_clf = OneVsRestClassifier(SVC())


In [73]:
ovr_clf.fit(X_train_scaled,y_train)

OneVsRestClassifier(estimator=SVC())

In [74]:
y_pred = ovr_clf.predict(X_test_scaled)

In [75]:
pd.crosstab(y_test,y_pred, rownames=['Classes réelles'], colnames=['Classes prédites'])

Classes prédites,1,2,3
Classes réelles,,,
1,105,18,3
2,42,35,3
3,44,31,4


- SGD Classifier

In [76]:
from sklearn.linear_model import SGDClassifier

In [77]:
sgd_clf = SGDClassifier()

In [78]:
sgd_clf.fit(X_train_scaled,y_train)

SGDClassifier()

In [79]:
y_pred = sgd_clf.predict(X_test_scaled)

In [80]:
pd.crosstab(y_test,y_pred, rownames=['Classes réelles'], colnames=['Classes prédites'])

Classes prédites,1,2,3
Classes réelles,,,
1,66,32,28
2,32,33,15
3,35,25,19
